# Tune GPT2 to generate controlled sentiment reviews
> Optimise GPT2 to produce IMDB movie reviews with controlled sentiment using a BERT sentiment classifier for rewards.

**WARNING:** We often experienced loss spikes in this examples which caused model training to fail or slow down. There is a [GitHub issue](https://github.com/lvwerra/trl/issues/101) to track the issue.

<div style="text-align: center">
<img src='https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/gpt2-ctrl-training-setup.png' width='600'>
<p style="text-align: center;"> <b>Figure:</b> Experiment setup to tune GPT2. The yellow arrows are outside the scope of this notebook, but the trained models are available through Hugging Face. </p>
</div>


The experiment setup is very similar to the positive sentiment notebook. However, in this notebook we fine-tune GPT2 (small) to generate **controlled** movie reviews based on the IMDB dataset. The model gets the target sentiment and 5 tokens from a real review and is tasked to produce continuations with the targeted sentiment. The reward for the continuations is calculated with the logits of a BERT sentiment classifier. That reward is then used for PPO training.

## Setup experiment

In [ ]:
!pip install -q wandb transformers
!pip install -U -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git
!pip install -q trl

### Import dependencies

In [3]:
%load_ext autoreload
%autoreload 2

In [8]:
import random
import torch
import wandb
import time
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
from random import choices
import matplotlib.pyplot as plt

tqdm.pandas()

from datasets import load_dataset

from transformers import AutoTokenizer, pipeline

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead, create_reference_model

### Configuration

In [65]:
sentiment_pipe_kwargs = {"top_k": None, "function_to_apply": "none"}

config = PPOConfig(
    model_name="lvwerra/gpt2-imdb", steps=51200, learning_rate=1.41e-5, remove_unused_columns=False, log_with="wandb",
    batch_size=4
)

txt_in_len = 5
txt_out_len = 20
seed = 1

In [10]:
np.random.seed(seed)

You can see that we load a GPT2 model called `gpt2_imdb`. This model was additionally fine-tuned on the IMDB dataset for 1 epoch with the huggingface [script](https://github.com/huggingface/transformers/blob/master/examples/run_language_modeling.py) (no special settings). The other parameters are mostly taken from the original paper ["Fine-Tuning Language Models from Human Preferences"](
https://arxiv.org/pdf/1909.08593.pdf). This model as well as the BERT model is available in the Huggingface model zoo [here](https://huggingface.co/models). The following code should automatically download the models.

## Load data and models

### Load pre-trained GPT2 language models

We load the GPT2 model with a value head and the tokenizer. We load the model twice; the first model is optimized while the second model serves as a reference to calculate the KL-divergence from the starting point. This serves as an additional reward signal in the PPO training to make sure the optimized model does not deviate too much from the original language model.

In [11]:
gpt2_model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
gpt2_model_ref = create_reference_model(gpt2_model)
gpt2_tokenizer = AutoTokenizer.from_pretrained(config.model_name)
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/577 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/548M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

### Load IMDB dataset
The IMDB dataset contains 50k movie review annotated with "positive"/"negative" feedback indicating the sentiment.  We load the IMDB dataset into a DataFrame and filter for comments that are at least 500 characters long and take the first 1000 characters of each comment. The first filter we apply to avoid comments that are less than `txt_in_len` token long and the second to avoid tokenizing way more text than we actually need.

In [56]:
# create the dataset
#
dataset = load_dataset("imdb", split="train")
dataset = dataset.rename_columns({"text": "review", "label": "sentiment"})
# make sure the comments are are at least 500 and trim to 1000
dataset = dataset.filter(lambda x: len(x["review"]) > 500, batched=False)
dataset = dataset.map(lambda x: {"review": x["review"][:1000]}, batched=False)
dataset

Dataset({
    features: ['review', 'sentiment'],
    num_rows: 22578
})

### Tokenize IMDB reviews

We tokenize all IMDB in advance to avoid tokenizing twice. In the first step we encode the queries and slice the first `txt_in_len` tokens. In a second step we decode these tokens back to text for later display.

In [57]:
dataset = dataset.map(
    lambda x: {"input_ids": gpt2_tokenizer.encode(" " + x["review"], return_tensors="pt")[0, :txt_in_len]},
    batched=False,
)
dataset = dataset.map(lambda x: {"query": gpt2_tokenizer.decode(x["input_ids"])}, batched=False)
dataset = dataset[:2000]
dataset

Map:   0%|          | 0/22578 [00:00<?, ? examples/s]

Map:   0%|          | 0/22578 [00:00<?, ? examples/s]

{'review': ['I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far

In [58]:
from datasets import Dataset
dataset = Dataset.from_dict(dataset)
dataset.set_format("pytorch")
dataset

Dataset({
    features: ['review', 'sentiment', 'input_ids', 'query'],
    num_rows: 2000
})

In [59]:
dataset[3]

{'review': "This film was probably inspired by Godard's Masculin, féminin and I urge you to see that film instead.<br /><br />The film has two strong elements and those are, (1) the realistic acting (2) the impressive, undeservedly good, photo. Apart from that, what strikes me most is the endless stream of silliness. Lena Nyman has to be most annoying actress in the world. She acts so stupid and with all the nudity in this film,...it's unattractive. Comparing to Godard's film, intellectuality has been replaced with stupidity. Without going too far on this subject, I would say that follows from the difference in ideals between the French and the Swedish society.<br /><br />A movie of its time, and place. 2/10.",
 'sentiment': tensor(0),
 'input_ids': tensor([ 770, 2646,  373, 2192, 7867]),
 'query': ' This film was probably inspired'}

In [60]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

In [67]:
ppo_trainer = PPOTrainer(config, gpt2_model, gpt2_model_ref, gpt2_tokenizer, dataset, data_collator=collator)
ppo_trainer

### Load BERT classifier
We load a BERT classifier fine-tuned on the IMDB dataset.

In [30]:
if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if torch.cuda.is_available() else "cpu"  # to avoid a `pipeline` bug
else:
    device = ppo_trainer.accelerator.device
print(device)

sentiment_pipe = pipeline("sentiment-analysis", "lvwerra/distilbert-imdb", device=device)

0


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

The model outputs are the logits for the negative and positive class. We will use the logits for positive class as a reward signal for the language model.

In [31]:
sentiment_pipe_kwargs

{'top_k': None, 'function_to_apply': 'none'}

In [32]:
text = "this movie was really bad!!"
output = sentiment_pipe(text, **sentiment_pipe_kwargs)
output

[{'label': 'NEGATIVE', 'score': 2.3350484371185303},
 {'label': 'POSITIVE', 'score': -2.726576328277588}]

In [33]:
text = "this movie was really good!!"
output = sentiment_pipe(text, **sentiment_pipe_kwargs)
output

[{'label': 'POSITIVE', 'score': 2.557040214538574},
 {'label': 'NEGATIVE', 'score': -2.294790267944336}]

In [38]:
text = "this movie was a documentary"
output = sentiment_pipe(text, top_k=5)
output

[{'label': 'POSITIVE', 'score': 0.8270527124404907},
 {'label': 'NEGATIVE', 'score': 0.17294730246067047}]

The resulting reward signal:

In [39]:
def extract_pipe_output(outputs):
    positive_logits = []
    for out in outputs:
        for element in out:
            if element["label"] == "POSITIVE":
                positive_logits.append(torch.tensor(element["score"]))
    return positive_logits

In [41]:
output[1]["score"]

0.17294730246067047

### Control token dict
We will append the control token at the beginning of each query to signal the model what the target sentiment is. Each control sequence consists of three tokens:

In [42]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # this should be handled by accelerate
device

device(type='cuda')

In [43]:
ctrl_str = ["[negative]", "[neutral]", "[positive]"]
ctrl_tokens = dict((s, gpt2_tokenizer.encode(s, return_tensors="pt").squeeze().to(device)) for s in ctrl_str)
ctrl_tokens

{'[negative]': tensor([   58, 31591,    60], device='cuda:0'),
 '[neutral]': tensor([   58, 29797,    60], device='cuda:0'),
 '[positive]': tensor([   58, 24561,    60], device='cuda:0')}

### Reward function

In [44]:
def pos_logit_to_reward(logit, task):
    """
    Process batch of positive logits according to their corresponding task.
    Take the positive sentiment logit and scale it for the task.
        task [negative]: reward = -logit
        task [neutral]: reward = -2*abs(logit)+4
        task [positive]: reward = logit
    """
    for i in range(len(logit)):
        if task[i] == "[negative]":
            logit[i] = -logit[i]
        elif task[i] == "[neutral]":
            logit[i] = -2 * torch.abs(logit[i]) + 4
        elif task[i] == "[positive]":
            pass
        else:
            raise ValueError("task has to be in [0, 1, 2]!")
    return logit

The following examples show the rewards for the cases where the classifier logit is 4, -4 and 0 for the three targets `['negative]`, `['neutral]` and `['positive']`. The scaling is not perfect as it differs between neutral and the other two classes. This is something to further investigate in the future. Ideally, one would use the logit output for each class individually, but since there is no dedicated class for neutral this is a workaround.

In [45]:
print(ctrl_str)

['[negative]', '[neutral]', '[positive]']


In [46]:
pos_logit_to_reward(torch.Tensor([4, 4, 4]), ctrl_str)

tensor([-4., -4.,  4.])

In [47]:
pos_logit_to_reward(torch.Tensor([-4, -4, -4]), ctrl_str)

tensor([ 4., -4., -4.])

In [48]:
pos_logit_to_reward(torch.Tensor([0, 0, 0]), ctrl_str)

tensor([-0., 4., 0.])

In [51]:
pos_logit_to_reward(torch.Tensor([1, 3, 4]), ctrl_str)

tensor([-1., -2.,  4.])

### Generation settings

In [54]:
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": gpt2_tokenizer.eos_token_id,
    "max_new_tokens": txt_out_len,
    "eos_token_id": -1,
}

## Optimize model

**Steps**

The training loop consists of the following steps:
1. Get a batch of queries and create random controls
2. Get the query responses from the policy
3. Join query and responses and tokenize for BERT analysis
4. Get sentiments for query/responses from BERT
5. Optimize policy with PPO using the (query, response, reward) triplet
6. Log all the training statistics

**Training time**

This step takes **~2h** on a P6000 GPU with the above specified settings.

## Process single batch (just for visualizing what's going on...)

In [68]:
b = next(iter(ppo_trainer.dataloader))
print(b)

{'review': ["This movie was exactly what I expected it to be when i first read the casting. I probably could have written a more exciting plot, it's a pity that they left it to a pack of Howler Monkeys. Alberto Tomba was surely a good skier but he has to thank God (and we too) that he does not have to rely on his actor skills to earn his living. He can't play, he can't talk, he can't even move very good on mainland without his skis... Michelle Hunziker is a pretty blonde girl, and that's all. She obviously wasn't chosen for her astounding competence in dramatic roles but most probably for her nice legs. Nevertheless I must admit that she could be the Tomba's acting teacher, because he's even a worse actor than her, and that's funny, especially considering that she isn't italian. I laughed all the time, watching this movie. I found it so ridiculous and meaningless that it actually made me laugh, loud, very loud.", 'This film, like the first one ("The Man From Snowy River") has the same 

In [74]:
for k in b.keys():
  print(k, ':', b[k][0])

review : This movie was exactly what I expected it to be when i first read the casting. I probably could have written a more exciting plot, it's a pity that they left it to a pack of Howler Monkeys. Alberto Tomba was surely a good skier but he has to thank God (and we too) that he does not have to rely on his actor skills to earn his living. He can't play, he can't talk, he can't even move very good on mainland without his skis... Michelle Hunziker is a pretty blonde girl, and that's all. She obviously wasn't chosen for her astounding competence in dramatic roles but most probably for her nice legs. Nevertheless I must admit that she could be the Tomba's acting teacher, because he's even a worse actor than her, and that's funny, especially considering that she isn't italian. I laughed all the time, watching this movie. I found it so ridiculous and meaningless that it actually made me laugh, loud, very loud.
sentiment : tensor(0, device='cuda:0')
input_ids : tensor([ 770, 3807,  373, 34

In [78]:

#### prepend a random control token
task_list = choices(ctrl_str, k=config.batch_size)
game_data["query"] = [t + q for t, q in zip(task_list, batch["query"])]
query_tensors = [torch.cat((ctrl_tokens[t], input_ids)) for t, input_ids in zip(task_list, batch["input_ids"])]
print(game_data)
print(query_tensors)

{'query': ['[positive] This show is painful to', "[positive] I don't hand out", '[neutral] The Darkling was a', '[negative] Growing up, Joe Str'], 'response': [' watch with emotions that are highly excite-reseemed. And a part-time, more', ' my highest (19) for nothing I can say for what they release.<|endoftext|> This is awesome,', ' real shocker, frick Barack (1970s) right and War Machine was a bit earlier period', "ider is a very funny and funny independent man who wrote some wonderful movies for TV's own screen,", ' out (I think it was around the second film before) he would get his first win. He', "' is a love story, but I don't think I knew how to say that the least-", ' advanced screen time when full help saw film.<br /><br />Boys saw one of the', ' ahead when people finish this movie! ;) <br /><br />Now that is my second rating', ' was disinterred from being focused on the political issue between ideologies.<br /><br />The', " this movie remembered for their acting, where the fi

In [88]:
#### get response from gpt2
response_tensors = []
for query in query_tensors:
    response = ppo_trainer.generate(query, **generation_kwargs)
    print(response)
    response_tensors.append(response[-txt_out_len:])

game_data["response"] = [gpt2_tokenizer.decode(r.squeeze()) for r in response_tensors]
game_data

tensor([[   58, 24561,    60,   770,   905,   318, 12132,   284,  2342,    13,
           887,   991,    11,  4045,    11,   845,   922,    13,   314,  2883,
           340,    13,   383,  8786,   547,   510,   612,   290]],
       device='cuda:0')
tensor([[   58, 24561,    60,   314,   836,   470,  1021,   503,   616,  7475,
            11,   475,   314,  1254,   922,    13,   314,  1053,  1682, 13818,
          1327,   618,   314,  6939, 12680,   318,  1679,  7225]],
       device='cuda:0')
tensor([[   58, 29797,    60,   383,  3801,  1359,   373,   257,  4622,  4785,
          1517,   329,   326,  1517,    11,   475,   339,   373,   257,  4159,
          2928,   319,   262,  3807,   290,   319,  1811,   661]],
       device='cuda:0')
tensor([[   58, 31591,    60, 37487,   510,    11,  5689,  4285,   624,  1044,
           290, 11000,  1842,   329,   257,   530,    12,  1659,    12,    64,
            12,    64,    12, 13571, 30248,    12,    79,  6197]],
       device='cuda:0')


{'query': ['[positive] This show is painful to',
  "[positive] I don't hand out",
  '[neutral] The Darkling was a',
  '[negative] Growing up, Joe Str'],
 'response': ['[positive] This show is painful to watch. But still, overall, very good. I enjoy it. The writers were up there and',
  "[positive] I don't hand out my letters, but I feel good. I've actually laughed hard when I realized THIS is 25%.",
  '[neutral] The Darkling was a slightly worse thing for that thing, but he was a minor impact on the movie and on several people',
  '[negative] Growing up, Joe Strickland and Marine love for a one-of-a-a-story lame-poster']}

In [89]:
#### sentiment analysis
texts = [q + r for q, r in zip(batch["query"], game_data["response"])]
logits = extract_pipe_output(sentiment_pipe(texts, **sentiment_pipe_kwargs))
rewards = pos_logit_to_reward(logits, task_list)
print(texts)
print(logits)
print(rewards)

[' This show is painful to[positive] This show is painful to watch. But still, overall, very good. I enjoy it. The writers were up there and', " I don't hand out[positive] I don't hand out my letters, but I feel good. I've actually laughed hard when I realized THIS is 25%.", ' The Darkling was a[neutral] The Darkling was a slightly worse thing for that thing, but he was a minor impact on the movie and on several people', ' Growing up, Joe Str[negative] Growing up, Joe Strickland and Marine love for a one-of-a-a-story lame-poster']
[tensor(2.8221), tensor(2.0462), tensor(1.9797), tensor(1.4448)]
[tensor(2.8221), tensor(2.0462), tensor(1.9797), tensor(1.4448)]


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


## Full Training

In [111]:
for epoch in range(2):
    for batch in tqdm(ppo_trainer.dataloader):
        (logs, game_data,) = (
            dict(),
            dict(),
        )

        #### prepend a random control token
        task_list = choices(ctrl_str, k=config.batch_size)
        game_data["query"] = [t + q for t, q in zip(task_list, batch["query"])]
        query_tensors = [torch.cat((ctrl_tokens[t], input_ids)) for t, input_ids in zip(task_list, batch["input_ids"])]

        #### get response from gpt2
        response_tensors = []
        for query in query_tensors:
            response = ppo_trainer.generate(query, **generation_kwargs)
            response_tensors.append(response.squeeze()[-txt_out_len:])
        game_data["response"] = [gpt2_tokenizer.decode(r.squeeze()) for r in response_tensors]

        #### sentiment analysis by a reward model (a pre-trained sentiment analyzer)
        texts = [q + r for q, r in zip(batch["query"], game_data["response"])]
        # Acc. to FAQs, it could be that sentiment analyzer mostly focuses on the "query" part.
        # in these cases, might hav eto normlaize like: reward(query + response) - reward(query)
        logits = extract_pipe_output(sentiment_pipe(texts, **sentiment_pipe_kwargs))
        rewards = pos_logit_to_reward(logits, task_list)

        #### Run PPO training
        t = time.time()
        stats = ppo_trainer.step(query_tensors, response_tensors, rewards)

        for cs in ctrl_str:
            key = "env/reward_" + cs.strip("[]")
            stats[key] = np.mean([r.cpu().numpy() for r, t in zip(rewards, task_list) if t == cs])
        ppo_trainer.log_stats(stats, game_data, rewards)

  0%|          | 0/500 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
  0%|          | 1/500 [00:03<26:40,  3.21s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
  0%|          | 2/500 [00:06<29:16,  3.53s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using

### Training progress

One can observe how the model starts to generate more positive outputs after a few optimisation steps.

> Note: Investigating the KL-divergence will probably show that at this point the model has not converged to the target KL-divergence, yet. To get there would require longer training or starting with a higher inital coefficient.

## Model inspection

### Reward distribution
First, we can have a look at the reward distribution. Both the negative and positive rewards are clearly shifted to high rewards. The neutral rewards, however, are still centered around zero. There are a few possible explanations for this. There could be a bug in the code and the way the neutral rewards are calculated. Another problem could be that sentence sometimes start with a strong sentiment and it is hard for the model shift the sentiment towards neutral.

In [ ]:
for ctrl_s in ctrl_str:
    plt.hist(
        [r for r, t in zip(logs["env/reward_dist"], task_list) if t == ctrl_s], density=True, alpha=0.5, label=ctrl_s
    )
plt.legend(loc="best")
plt.title("reward distribution")
plt.grid(True)
plt.show()

## Save model
Finally, we save the model to disk for later usage.

In [113]:
model_id = 'shahzebnaveed/gpt2-sentiment-rlhf'
gpt2_model.save_pretrained(model_id)
gpt2_tokenizer.save_pretrained(model_id)

('gpt2-imdb-ctrl/tokenizer_config.json',
 'gpt2-imdb-ctrl/special_tokens_map.json',
 'gpt2-imdb-ctrl/vocab.json',
 'gpt2-imdb-ctrl/merges.txt',
 'gpt2-imdb-ctrl/added_tokens.json',
 'gpt2-imdb-ctrl/tokenizer.json')

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
gpt2_model.push_to_hub(model_id, use_auth_token=True)